In [ ]:
#default_exp asyncUtil

# async
> tools to help writing async python codes

In [ ]:
#hide
from nbdev.showdoc import *

# async wrap

In [ ]:
#export
import asyncio
from functools import wraps, partial

def async_wrap(func):
    @wraps(func)
    async def run(*args, loop=None, executor=None, **kwargs):
        if loop is None:
            loop = asyncio.get_event_loop()
        pfunc = partial(func, *args, **kwargs)
        return await loop.run_in_executor(executor, pfunc)
    return run


In [ ]:
%%time
@async_wrap
def aSlowFunc(input_:str):
  time.sleep(2)
  return input_

## async func execute
import nest_asyncio, time
nest_asyncio.apply()
async def runASlowFunc(input_):
  return await aSlowFunc(input_)
async def runLoop():
  rtup = (runASlowFunc(i) for i in range (10))
  r = await asyncio.gather(*rtup)
  return r
asyncio.run(runLoop())

CPU times: user 7.07 ms, sys: 0 ns, total: 7.07 ms
Wall time: 4.01 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# thread mapping

In [ ]:
#export
import multiprocessing.dummy
from typing import Callable, List, Any, Iterable
from beartype import beartype
@beartype
def asyncMap(f:Callable, data:Iterable[Any], threads:int = 5)->Any:
  p = multiprocessing.dummy.Pool(threads)
  return p.map(f,data)

In [ ]:
%%time
import time
asyncMap(lambda x: (x+1, time.sleep(1))[0] , range(100), threads = 100)[:10]

CPU times: user 25.2 ms, sys: 1 ms, total: 26.2 ms
Wall time: 1.02 s


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
def aSlowFunc(x):
  time.sleep(1)
  return x

In [ ]:
%%time
asyncMap(aSlowFunc, range(100))[:10]

CPU times: user 4.57 ms, sys: 4.95 ms, total: 9.52 ms
Wall time: 20 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
input_ = list(zip(range(10), range(1,11)))
print(input_)
asyncMap(lambda x: (lambda x,y: x+y )(x[0],x[1]), input_)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10)]


[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

# asyncAwaitMap

In [ ]:
#export
def asyncAwaitMap(f:Callable, data:Iterable[Any])->Any:
  af = async_wrap(f) # convert to async func
  async def runLoop():
    rtup = (af(i) for i in data)
    return await asyncio.gather(*rtup)
  return asyncio.run(runLoop())

In [ ]:
%%time
import nest_asyncio
nest_asyncio.apply()
asyncAwaitMap(aSlowFunc, range(100))[:10]

CPU times: user 22.2 ms, sys: 7.76 ms, total: 30 ms
Wall time: 17 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
input_ = list(zip(range(10), range(1,11)))
print(input_)
asyncAwaitMap(lambda x: (lambda x,y: x+y )(x[0],x[1]), input_)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10)]


[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

# AsyncThread

In [ ]:
#export
from concurrent.futures import ThreadPoolExecutor
def asyncThreadMap(f,data, threads=10):
  with ThreadPoolExecutor(threads) as tr:
    return tr.map(f,data)

In [ ]:
%%time
def aSlowFunc(x):
  time.sleep(1)
  return x

list(asyncThreadMap(aSlowFunc, range(100)))[:10]

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 10 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# AsyncProcess map

In [ ]:
#export
from concurrent.futures import ProcessPoolExecutor
def asyncProcessMap(f,data, threads=10):
  with ProcessPoolExecutor(threads) as tr:
    return tr.map(f,data)

In [ ]:
%%time
def aSlowFunc(x):
  time.sleep(1)
  return x

list(asyncProcessMap(aSlowFunc, range(100)))[:10]

CPU times: user 30.2 ms, sys: 54.3 ms, total: 84.5 ms
Wall time: 10.1 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]